# Quarterly Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Q3: October, November, December

### Notes:
- Q3 2020 excel sheet does not contain totals for the 'UNKNOWN/NONCLASSIFIABLE' category; respective column for 2019 was removed

In [1]:
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Q3 2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-03-quarterly-sales.xlsx'
q3_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q3_2020.drop(q3_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
q3_2020['County'] = q3_2020['County'].fillna(method='ffill')
q3_2020['Location Code'] = q3_2020['Location Code'].fillna(method='ffill')
q3_2020['City'] = q3_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q3_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q3_2020['Rounded Up'] = q3_2020['Rounded Up'].map(crosswalk)

q3_2020.head()


,County,Location Code,City,Economic Sector (NAICS Code),2020Q3,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",80000.0,True
1,Beaver County,1002.0,Beaver City,UTILITIES (22),700000.0,True
2,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),150000.0,True
3,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),1880967.0,False
4,Beaver County,1002.0,Beaver City,WHOLESALE TRADE-DURABLE GOODS (423),310250.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_q3_2020 = pd.pivot_table(q3_2020,values = '2020Q3',index ='City', columns = 'Economic Sector (NAICS Code)', 
                                 aggfunc='first')

# fill NAs with 0
sectors_q3_2020 = sectors_q3_2020.fillna(0)

In [6]:
sectors_q3_2020.columns

Index(['ACCOMMODATION (721)',
       'ADMIN. & SUPPORT & WASTE MANAG. & REMED. SERVICES (56)',
       'AGRICULTURE, FORESTRY, FISHING & HUNTING (11)',
       'ARTS, ENTERTAINMENT AND RECREATION (71)', 'CONSTRUCTION (23)',
       'EDUCATIONAL SERVICES (61)', 'FINANCE & INSURANCE (52)',
       'FOOD SERVICES & DRINKING PLACES (722)',
       'HEALTH CARE & SOCIAL ASSISTANCE (62)', 'INFORMATION (51)',
       'MANAGEMENT OF COMPANIES & ENTERPRISES (55)', 'MANUFACTURING (31-33)',
       'MINING, QUARRYING, & OIL & GAS EXTRACTION (21)',
       'OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81)',
       'PRIOR-PERIOD PAYMENTS & REFUNDS', 'PRIVATE MOTOR VEHICLE SALES',
       'PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES (54)',
       'PUBLIC ADMINISTRATION (92)', 'REAL ESTATE, RENTAL & LEASING (53)',
       'RETAIL-BUILD. MATERIAL, GARDEN EQUIP. & SUPPLIES DEALERS (444)',
       'RETAIL-CLOTHING & CLOTHING ACCESSORIES STORES (448)',
       'RETAIL-ELECTRONICS & APPLIANCE STORES (443)',
       

In [7]:
# fix column names
#new_names = [replace(item) for item in list(sectors_q3_2020.columns)]
new_names = ['L_ACCOMMODATION_20', 'S_ADMIN_SUPPORT_20','A_AG_WILDLIFE_20','L_CULTURAL_REC_20','C_CONSTRUCTION_20',
'E_EDUCATION_20','S_FINANCIAL_20','L_RSTRNT_BAR_20','H_HEALTH_CARE_20','S_IT_DATA_20','S_CORPORATE_MGMT_20',
'M_MANUFACTURING_20','E_EXTRACTION_20','O_OTHER_20','X_ADJUSTMENTS_20','R_AUTO_PRIVATE_20','S_PROF_TECH_SRV_20',
'G_GOVERNMENT_20','S_RENTAL_LEASING_20','R_BUILDING_SUPPLY_20','R_CLOTHING_20','R_ELECTRONICS_20','R_GROCERY_BEV_20',
'R_FURNITURE_20','R_GAS_STATIONS_20','R_GENERAL_RETAIL_20','R_HEALTH_RETAIL_20','R_OTHER_RETAIL_20','R_AUTO_RETAIL_20',
'R_NONSTORE_RETAIL_20','R_SPORT_HOBBY_20','L_SPECIAL_EVENT_20','W_DISTRIBUTION_20','U_UTILITIES_20',
'W_WHLSALE_DURABLE_20','S_WHLSLE_ETRADE_20','W_WHLSLE_NDURABLE_20']
sectors_q3_2020.columns = new_names

# get the total sales tax revenue
sectors_q3_2020['TOTAL_20'] = sectors_q3_2020[new_names].sum(axis=1)

sectors_q3_2020.head()

,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,610610.0,684390.0,700000.0,1742593.0,9050703.0,196272.0,2320119.0,27458710.0,409886.0,7291812.0,2000.0,3371140.0,80000.0,12896236.0,1169425.0,5500000.0,4721426.0,45000.0,4239640.0,24434022.0,8831414.0,6888175.0,4204031.0,4882872.0,3250000.0,46130407.0,6456783.0,8120283.0,114645141.0,17851031.0,9512346.0,30000.0,610210.0,7250000.0,15721518.0,603981.0,7912279.0,369824455.0
Beaver City,1796700.0,22137.0,80000.0,7000.0,150000.0,6000.0,487846.0,3207140.0,60000.0,470173.0,1000.0,1880967.0,0.0,1165909.0,218144.0,500000.0,141782.0,0.0,161945.0,958291.0,126941.0,30764.0,2250000.0,3577.0,4750000.0,489849.0,224718.0,264052.0,1046582.0,1487748.0,56056.0,0.0,8000.0,700000.0,310250.0,52849.0,140450.0,23256870.0
Blanding,641588.0,63792.0,1000.0,40000.0,150000.0,150000.0,18431.0,450000.0,40000.0,454379.0,3000.0,128084.0,1000.0,697047.0,225818.0,900000.0,139751.0,400000.0,59602.0,2580370.0,108254.0,107663.0,2000000.0,10512.0,1750000.0,656475.0,67668.0,120904.0,406840.0,1479771.0,96572.0,0.0,8000.0,150000.0,353949.0,7889.0,301054.0,14769413.0
Bountiful,200000.0,1528237.0,2971.0,1098401.0,1976421.0,143845.0,1263042.0,13021850.0,1229667.0,5588283.0,25000.0,2579458.0,40000.0,6665067.0,3441209.0,6000000.0,3618011.0,800000.0,2180497.0,1557832.0,3394995.0,4258239.0,22292365.0,1913518.0,2750000.0,2294599.0,2311722.0,3846816.0,54387174.0,18021178.0,2166578.0,15000.0,15000.0,8000000.0,5685595.0,119584.0,708373.0,185140527.0
Brigham,1237268.0,198096.0,3000.0,700000.0,611762.0,54266.0,293778.0,7589533.0,349797.0,2658038.0,8000.0,7690680.0,6000.0,3747382.0,1276922.0,3250000.0,764659.0,6000.0,1905406.0,1876175.0,534765.0,1315595.0,11700758.0,535790.0,3750000.0,676166.0,471740.0,1010190.0,18206008.0,5995719.0,456584.0,1000.0,70000.0,3250000.0,5419024.0,86964.0,305138.0,88012203.0


### Load Q3 2019 sales tax revenue data

In [8]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-03-quarterly-sales.xlsx'
q3_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q3_2019.drop(q3_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
q3_2019['County'] = q3_2019['County'].fillna(method='ffill')
q3_2019['Location Code'] = q3_2019['Location Code'].fillna(method='ffill')
q3_2019['City'] = q3_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q3_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q3_2019['Rounded Up'] = q3_2019['Rounded Up'].map(crosswalk)

q3_2019.tail()

,County,Location Code,City,Economic Sector (NAICS Code),2019Q3,Rounded Up
2408,Weber County,29040.0,South Ogden,FOOD SERVICES & DRINKING PLACES (722),9854548.0,False
2409,Weber County,29040.0,South Ogden,OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81),1663672.0,False
2410,Weber County,29040.0,South Ogden,PUBLIC ADMINISTRATION (92),7000.0,True
2411,Weber County,29040.0,South Ogden,PRIVATE MOTOR VEHICLE SALES,1250000.0,True
2412,Weber County,29040.0,South Ogden,PRIOR-PERIOD PAYMENTS & REFUNDS,532896.0,False


In [9]:
# pivot the table to get sectors as columns with tax values by city
sectors_q3_2019 = pd.pivot_table(q3_2019,values = '2019Q3',index ='City', columns = 'Economic Sector (NAICS Code)', 
                                 aggfunc='first')

# fill NAs with 0
sectors_q3_2019 = sectors_q3_2019.fillna(0)

# fix column names
#new_names = [replace(item) for item in list(sectors_q3_2019.columns)]
new_names = ['L_ACCOMMODATION_19', 'S_ADMIN_SUPPORT_19','A_AG_WILDLIFE_19','L_CULTURAL_REC_19','C_CONSTRUCTION_19',
'E_EDUCATION_19','S_FINANCIAL_19','L_RSTRNT_BAR_19','H_HEALTH_CARE_19','S_IT_DATA_19','S_CORPORATE_MGMT_19',
'M_MANUFACTURING_19','E_EXTRACTION_19','O_OTHER_19','X_ADJUSTMENTS_19','R_AUTO_PRIVATE_19','S_PROF_TECH_SRV_19',
'G_GOVERNMENT_19','S_RENTAL_LEASING_19','R_BUILDING_SUPPLY_19','R_CLOTHING_19','R_ELECTRONICS_19','R_GROCERY_BEV_19',
'R_FURNITURE_19','R_GAS_STATIONS_19','R_GENERAL_RETAIL_19','R_HEALTH_RETAIL_19','R_OTHER_RETAIL_19','R_AUTO_RETAIL_19',
'R_NONSTORE_RETAIL_19','R_SPORT_HOBBY_19','L_SPECIAL_EVENT_19','W_DISTRIBUTION_19','X_UNKNOWN_19','U_UTILITIES_19',
'W_WHLSALE_DURABLE_19','S_WHLSLE_ETRADE_19','W_WHLSLE_NDURABLE_19']
sectors_q3_2019.columns = new_names

# drop unknown column since its not present in 2020
sectors_q3_2019.drop('X_UNKNOWN_19', axis=1, inplace=True)
new_names.remove('X_UNKNOWN_19')

# get the total sales tax revenue
sectors_q3_2019['TOTAL_19'] = sectors_q3_2019[new_names].sum(axis=1)

sectors_q3_2019.head()

,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,800000.0,668822.0,350000.0,1644861.0,8342718.0,169050.0,1537569.0,24349796.0,388483.0,9566264.0,15000.0,2918859.0,25000.0,11162595.0,2294792.0,3250000.0,8069449.0,35000.0,4118002.0,19753584.0,8371989.0,7249934.0,4203317.0,3780567.0,3062233.0,42360207.0,5681087.0,6232059.0,99475031.0,8651524.0,7781573.0,450000.0,553684.0,8250000.0,12356108.0,28074.0,6163163.0,324110394.0
Beaver City,1486864.0,27631.0,60000.0,8000.0,200000.0,20000.0,256761.0,2805382.0,50000.0,440226.0,2000.0,1582128.0,0.0,1009688.0,222285.0,400000.0,114258.0,0.0,101136.0,702714.0,81107.0,29373.0,2000000.0,3000.0,4000000.0,478622.0,70037.0,167626.0,877288.0,708395.0,28704.0,2000.0,2000.0,600000.0,307233.0,20000.0,181556.0,19046014.0
Blanding,1500000.0,61062.0,1000.0,60000.0,203713.0,150000.0,17946.0,700000.0,70000.0,423908.0,1000.0,221758.0,0.0,283537.0,214830.0,900000.0,71006.0,200000.0,93547.0,1668222.0,72090.0,119245.0,2000000.0,12886.0,1500000.0,637511.0,53646.0,121127.0,364437.0,690609.0,121517.0,1000.0,15000.0,450000.0,292453.0,2000.0,151858.0,13446908.0
Bountiful,200000.0,2313211.0,2000.0,1042613.0,1657783.0,144399.0,1224894.0,13309802.0,1089477.0,4928417.0,35000.0,2503147.0,30000.0,6902004.0,1174422.0,4750000.0,2313666.0,700000.0,2361609.0,1155702.0,3015615.0,3470102.0,20434125.0,1705584.0,2750000.0,1793300.0,1684330.0,3002183.0,57813709.0,8280131.0,1622661.0,25000.0,60000.0,7500000.0,4556694.0,10000.0,603078.0,166164658.0
Brigham,2000000.0,172922.0,2000.0,800000.0,1482857.0,60540.0,203754.0,7103067.0,509334.0,2728484.0,4000.0,8387846.0,10000.0,3116643.0,1860373.0,2750000.0,452784.0,6000.0,1683923.0,1607103.0,434641.0,1100882.0,10088650.0,510411.0,3250000.0,553762.0,387537.0,937747.0,15304834.0,3014300.0,372881.0,450000.0,70000.0,3000000.0,3827145.0,10000.0,324071.0,78578491.0


### Calculate the differences 
*2020 - 2019 = Diff*

In [10]:
# Check join output shape
print(sectors_q3_2019.shape)
print(sectors_q3_2020.shape)

(66, 38)
(66, 38)


In [11]:
# merge the 2019-2020 difference with the sdf
sectors_q3_diff = pd.DataFrame(sectors_q3_2020.values - sectors_q3_2019.values)

# add back the column names
new_names = ['L_ACCOMMODATION_D20', 'S_ADMIN_SUPPORT_D20','A_AG_WILDLIFE_D20','L_CULTURAL_REC_D20','C_CONSTRUCTION_D20',
'E_EDUCATION_D20','S_FINANCIAL_D20','L_RSTRNT_BAR_D20','H_HEALTH_CARE_D20','S_IT_DATA_D20','S_CORPORATE_MGMT_D20',
'M_MANUFACTURING_D20','E_EXTRACTION_D20','O_OTHER_D20','X_ADJUSTMENTS_D20','R_AUTO_PRIVATE_D20','S_PROF_TECH_SRV_D20',
'G_GOVERNMENT_D20','S_RENTAL_LEASING_D20','R_BUILDING_SUPPLY_D20','R_CLOTHING_D20','R_ELECTRONICS_D20','R_GROCERY_BEV_D20',
'R_FURNITURE_D20','R_GAS_STATIONS_D20','R_GENERAL_RETAIL_D20','R_HEALTH_RETAIL_D20','R_OTHER_RETAIL_D20','R_AUTO_RETAIL_D20',
'R_NONSTORE_RETAIL_D20','R_SPORT_HOBBY_D20','L_SPECIAL_EVENT_D20','W_DISTRIBUTION_D20','U_UTILITIES_D20',
'W_WHLSALE_DURABLE_D20','S_WHLSLE_ETRADE_D20','W_WHLSLE_NDURABLE_D20', 'TOTAL_D20']

sectors_q3_diff.columns = new_names

# add back cities as index
sectors_q3_diff['City'] = sectors_q3_2020.index
sectors_q3_diff = sectors_q3_diff.set_index('City')

#check table
sectors_q3_diff.head(10)

,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,-189390.0,15568.0,350000.0,97732.0,707985.0,27222.0,782550.0,3108914.0,21403.0,-2274452.0,-13000.0,452281.0,55000.0,1733641.0,-1125367.0,2250000.0,-3348023.0,10000.0,121638.0,4680438.0,459425.0,-361759.0,714.0,1102305.0,187767.0,3770200.0,775696.0,1888224.0,15170110.0,9199507.0,1730773.0,-420000.0,56526.0,-1000000.0,3365410.0,575907.0,1749116.0,45714061.0
Beaver City,309836.0,-5494.0,20000.0,-1000.0,-50000.0,-14000.0,231085.0,401758.0,10000.0,29947.0,-1000.0,298839.0,0.0,156221.0,-4141.0,100000.0,27524.0,0.0,60809.0,255577.0,45834.0,1391.0,250000.0,577.0,750000.0,11227.0,154681.0,96426.0,169294.0,779353.0,27352.0,-2000.0,6000.0,100000.0,3017.0,32849.0,-41106.0,4210856.0
Blanding,-858412.0,2730.0,0.0,-20000.0,-53713.0,0.0,485.0,-250000.0,-30000.0,30471.0,2000.0,-93674.0,1000.0,413510.0,10988.0,0.0,68745.0,200000.0,-33945.0,912148.0,36164.0,-11582.0,0.0,-2374.0,250000.0,18964.0,14022.0,-223.0,42403.0,789162.0,-24945.0,-1000.0,-7000.0,-300000.0,61496.0,5889.0,149196.0,1322505.0
Bountiful,0.0,-784974.0,971.0,55788.0,318638.0,-554.0,38148.0,-287952.0,140190.0,659866.0,-10000.0,76311.0,10000.0,-236937.0,2266787.0,1250000.0,1304345.0,100000.0,-181112.0,402130.0,379380.0,788137.0,1858240.0,207934.0,0.0,501299.0,627392.0,844633.0,-3426535.0,9741047.0,543917.0,-10000.0,-45000.0,500000.0,1128901.0,109584.0,105295.0,18975869.0
Brigham,-762732.0,25174.0,1000.0,-100000.0,-871095.0,-6274.0,90024.0,486466.0,-159537.0,-70446.0,4000.0,-697166.0,-4000.0,630739.0,-583451.0,500000.0,311875.0,0.0,221483.0,269072.0,100124.0,214713.0,1612108.0,25379.0,500000.0,122404.0,84203.0,72443.0,2901174.0,2981419.0,83703.0,-449000.0,0.0,250000.0,1591879.0,76964.0,-18933.0,9433712.0
Cedar City,-977142.0,672870.0,1831.0,5509.0,1230319.0,-410014.0,-613688.0,2201010.0,-3308.0,86260.0,5000.0,2089442.0,192000.0,949120.0,-1356016.0,2000000.0,754940.0,100000.0,31656.0,7575183.0,496436.0,434137.0,3366674.0,1582504.0,1163384.0,3482916.0,202336.0,2001681.0,2759166.0,7195804.0,2075259.0,-150000.0,161163.0,750000.0,3321739.0,88395.0,-329396.0,43137170.0
Centerville,15000.0,93924.0,0.0,12000.0,-272030.0,8549.0,-78639.0,297630.0,8725.0,-1379614.0,-45000.0,-1859938.0,1000.0,164707.0,1107353.0,1000000.0,838978.0,-6000.0,13615.0,2469605.0,28869.0,-229261.0,459137.0,268411.0,250000.0,317672.0,-30885.0,-485364.0,-339720.0,3635347.0,472979.0,192000.0,2000.0,0.0,749533.0,25194.0,-59247.0,7646530.0
Clearfield,0.0,105639.0,2000.0,-230044.0,168460.0,22699.0,9250.0,679235.0,-11354.0,599259.0,250000.0,-218758.0,100000.0,484191.0,777589.0,750000.0,1556133.0,0.0,978667.0,266776.0,302452.0,-93519.0,729547.0,28787.0,998833.0,284555.0,37958.0,1129427.0,89051.0,6395071.0,584289.0,-15000.0,30000.0,-1500000.0,763596.0,118027.0,-69013.0,16103803.0
Cottonwood Heights,27834.0,-155178.0,1000.0,-1720000.0,994406.0,48169.0,-948979.0,-1771987.0,-39100.0,885814.0,-200000.0,125153.0,-5000.0,115436.0,927228.0,3000000.0,2569929.0,-50000.0,-142803.0,2292987.0,304535.0,-2611387.0,2568652.0,68204.0,250000.0,5290984.0,764038.0,255307.0,136227.0,8001018.0,179241.0,-40000.0,-320000.0,500000.0,355443.0,101350.0,-24680.0,21733841.0


### Load municipalities and townships shapefile and join to formatted sales tax data

In [12]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [13]:
# merge the 2020 sdf with the pivoted data
cities_sdf2 = cities_sdf.merge(sectors_q3_2019, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q3_2020, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q3_diff, left_on= "NAME", right_on="City", how="inner")
cities_sdf2.tail(10)


,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
55,222,Syracuse,Cities,049,25113,30400,10.193900,"{'rings': [[[411406.60675760085, 4549994.30436...",45000.0,72936.0,1000.0,1250000.0,223496.0,28558.0,554255.0,4907056.0,70000.0,2700893.0,20000.0,334474.0,1000.0,1409940.0,1559458.0,2750000.0,828221.0,25000.0,805700.0,405103.0,410014.0,777058.0,8750000.0,10131489.0,500000.0,18060676.0,355592.0,570867.0,513866.0,4860346.0,125783.0,15000.0,2000.0,4750000.0,1163045.0,25000.0,198476.0,69201302.0,60000.0,93630.0,2000.0,1250000.0,1235199.0,27911.0,446604.0,5428189.0,80000.0,2661884.0,6000.0,2591357.0,15000.0,1554313.0,1357494.0,5250000.0,848124.0,2000.0,1185857.0,1579395.0,772689.0,1118869.0,10805857.0,5572633.0,600000.0,19473844.0,473610.0,623787.0,657463.0,11825614.0,299906.0,0.0,25000.0,5250000.0,2343234.0,116716.0,395541.0,86029720.0,15000.0,20694.0,1000.0,0.0,1011703.0,-647.0,-107651.0,521133.0,10000.0,-39009.0,-14000.0,2256883.0,14000.0,144373.0,-201964.0,2500000.0,19903.0,-23000.0,380157.0,1174292.0,362675.0,341811.0,2055857.0,-4558856.0,100000.0,1413168.0,118018.0,52920.0,143597.0,6965268.0,174123.0,-15000.0,23000.0,500000.0,1180189.0,91716.0,197065.0,16828418.0
56,224,Taylorsville,Cities,142,59989,60192,10.839700,"{'rings': [[[422191.826826254, 4504303.3642982...",450000.0,481676.0,9000.0,2042324.0,1613198.0,131432.0,2204654.0,28130157.0,110489.0,8563764.0,35000.0,3151106.0,2000.0,2411316.0,1391656.0,4500000.0,1754721.0,-2000.0,2078966.0,320446.0,6346141.0,1774980.0,14807119.0,505464.0,4500000.0,25816425.0,1103015.0,4731430.0,6149630.0,7088643.0,6302519.0,45000.0,40000.0,10250000.0,3460228.0,25000.0,422186.0,152747685.0,400000.0,726327.0,10464.0,2000000.0,1331706.0,105767.0,2262112.0,28620381.0,160768.0,6125919.0,10000.0,2356003.0,2000.0,2317794.0,3727004.0,5250000.0,2729777.0,25000.0,2210665.0,487013.0,5975435.0,2232900.0,14687747.0,504724.0,4750000.0,26564770.0,1524017.0,5822221.0,7266727.0,16633918.0,7138553.0,1000.0,40000.0,102500

In [14]:
# Confirm join count, Salt Lake County (Unincorporated) should be the only one not joined
l1 = list(q3_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

['Salt Lake County (Unincorporated)']

In [15]:
# Create file gdb and export sdf to feature class
outputs = '.\\Outputs'

gdb = os.path.join(outputs, "taxable_sales.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "taxable_sales.gdb")

cities_sdf2.spatial.to_featureclass(location=os.path.join(outputs, "taxable_sales.gdb","taxable_sales_utah_q3"))

'E:\\Projects\\UT_Sales_Tax_Analysis\\Outputs\\taxable_sales.gdb\\taxable_sales_utah_q3'

### Analysis

In [16]:
# Get year specific column names
standard_cols = ['fid','name','source','salestaxid','poplastcen','poplastcen','area_sqmi', 'SHAPE']
names_19 = standard_cols + [col for col in cities_sdf2.columns if '_19' in col]
names_20 = standard_cols + [col for col in cities_sdf2.columns if '_20' in col]
names_D20 = standard_cols + [col for col in cities_sdf2.columns if '_d20' in col]

# subset columns by time
cities_q3_19 = cities_sdf2[names_19]
cities_q3_20 = cities_sdf2[names_20]
cities_q3_d20 = cities_sdf2[names_D20]


In [17]:
# 10 cities with smallest difference of revenue
cities_q3_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=False).head(10)

,name,poplastcen,poplastcen,area_sqmi,total_d20
54,St George,76548,76548,77.209900,132030878.0
62,West Jordan,105891,105891,32.306999,90642198.0
34,Orem,92993,92993,18.628000,67079183.0
33,Ogden,85183,85183,27.493500,63764733.0
23,Logan,49612,49612,18.485800,63421246.0
48,South Jordan,51057,51057,22.263201,62723200.0
20,Layton,69457,69457,22.576200,61634561.0
51,Spanish Fork,36232,36232,16.249701,52951283.0
21,Lehi,49315,49315,28.898199,51286416.0
50,South Salt Lake,24282,24282,6.936550,51193410.0


In [18]:
# 10 cities with largest difference of revenue
cities_q3_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=True).head(10)

,name,poplastcen,poplastcen,area_sqmi,total_d20
45,Salt Lake City,190749,190749,110.734001,-139795789.0
12,Farmington,18752,18752,10.035100,-27566629.0
35,Park City,7709,7709,20.341700,-10722173.0
43,Roosevelt,6260,6260,6.757070,-10027223.0
25,Midvale,28644,28644,5.853220,-7935591.0
28,Monticello,2053,2053,3.349260,-1426458.0
32,North Salt Lake,16930,16930,8.512150,-431477.0
17,Kanab,4413,4413,14.473200,160837.0
59,Vernal,9460,9460,4.632710,1026651.0
2,Blanding,3514,3514,13.205500,1322505.0


#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)